In [1]:
from selenium.webdriver.common.action_chains import ActionChains
import os
from bs4 import BeautifulSoup
import requests
from splinter import Browser
import pandas as pd
from requests import Session
from bs4 import BeautifulSoup as bs
import re
import json
from pprint import pprint
from datetime import datetime, date, time, timedelta
from time import sleep
from selenium import webdriver
import config
from selenium.webdriver.support.ui import Select
import numpy as np
import re

In [2]:
now=datetime.now()
today=date.today()
today_m_d_y=today.strftime('%m%d%y')

In [3]:
df=pd.read_csv('/Users/jessicaetchechury/Desktop/JEtchCreationsDashboard/eRank - Dates.csv')


In [4]:
df.head()

,#,MainImage,ListingTitle,ListingID,SKU,OriginallyCreated,LastUpdated,ExpiryDate
0,15,NaN,"Bridal Shower Signs, Digital File, Travel Them...",691796412,NaN,2019-05-11 22:07,2019-07-04 18:10,2019-09-11 22:30
1,2,NaN,Wedding Centerpiece - Centerpiece - Engagement...,680069172,NaN,2019-03-26 21:57,2019-07-04 18:16,2019-07-27 00:10
2,9,NaN,"Wedding Centerpiece Package, Centerpiece, Enga...",680085112,NaN,2019-03-27 00:00,2019-07-04 18:17,2019-07-27 00:10
3,7,NaN,"Wedding Centerpiece Package, Centerpiece, Enga...",680082346,NaN,2019-03-26 23:34,2019-07-04 18:20,2019-07-27 00:10
4,20,NaN,"Centerpiece, Wedding, Engagement Party Centerp...",700382982,NaN,2019-06-14 18:35,2019-07-04 19:02,2019-10-14 18:41


In [5]:
df2=df[['ListingID','OriginallyCreated']]
df2.head()

,ListingID,OriginallyCreated
0,691796412,2019-05-11 22:07
1,680069172,2019-03-26 21:57
2,680085112,2019-03-27 00:00
3,680082346,2019-03-26 23:34
4,700382982,2019-06-14 18:35


In [6]:
df2['DateCreated'] = pd.to_datetime(df2.OriginallyCreated)
df2['DateCreated'] = df2['DateCreated'].dt.strftime('%Y-%m-%d')
df3=df2[['ListingID','DateCreated']]
df3.head()

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,ListingID,DateCreated
0,691796412,2019-05-11
1,680069172,2019-03-26
2,680085112,2019-03-27
3,680082346,2019-03-26
4,700382982,2019-06-14


In [7]:
df3 = df3.astype({"DateCreated":str})
df3.head()

,ListingID,DateCreated
0,691796412,2019-05-11
1,680069172,2019-03-26
2,680085112,2019-03-27
3,680082346,2019-03-26
4,700382982,2019-06-14


In [8]:
df3['s_year']=df3.DateCreated.str.split('-').str[0]
df3['s_month']=df3.DateCreated.str.split('-').str[1]
df3['s_day']=df3.DateCreated.str.split('-').str[2]

df3.head()

,ListingID,DateCreated,s_year,s_month,s_day
0,691796412,2019-05-11,2019,05,11
1,680069172,2019-03-26,2019,03,26
2,680085112,2019-03-27,2019,03,27
3,680082346,2019-03-26,2019,03,26
4,700382982,2019-06-14,2019,06,14


In [9]:
df3 = df3.astype({"s_year": int, "s_month": int,"s_day":int})
df3.dtypes

ListingID       int64
DateCreated    object
s_year          int64
s_month         int64
s_day           int64
dtype: object

In [10]:
tomorrow = datetime.now() + timedelta(days=1)
df3['inclusive_end_date']=tomorrow

df3['e_day'] = df3['inclusive_end_date'].dt.day
df3['e_month'] = df3['inclusive_end_date'].dt.month
df3['e_year'] = df3['inclusive_end_date'].dt.year

df3.head()

,ListingID,DateCreated,s_year,s_month,s_day,inclusive_end_date,e_day,e_month,e_year
0,691796412,2019-05-11,2019,5,11,2019-07-11 18:51:58.815869,11,7,2019
1,680069172,2019-03-26,2019,3,26,2019-07-11 18:51:58.815869,11,7,2019
2,680085112,2019-03-27,2019,3,27,2019-07-11 18:51:58.815869,11,7,2019
3,680082346,2019-03-26,2019,3,26,2019-07-11 18:51:58.815869,11,7,2019
4,700382982,2019-06-14,2019,6,14,2019-07-11 18:51:58.815869,11,7,2019


In [11]:
# d = {'listingID': ['701341356', '687898920'], 's_year': [2019,2019], 's_month': [6,6],
#     's_day': [24,24],'e_year': [2019,2019], 'e_month': [7,7],
#     'e_day': [10,10]}
# mock_df = pd.DataFrame(data=d)
# mock_df

In [12]:
unique_listings=df.ListingID.unique()

In [13]:
# Start Browser
driver = webdriver.Chrome('/usr/local/bin/chromedriver')
# Navigate to login page
driver.get('https://www.etsy.com/signin')
# Populate sign in fields
emailElem = driver.find_element_by_id('join_neu_email_field')
emailElem.send_keys(config.email_address)
passwordElem = driver.find_element_by_id('join_neu_password_field')
passwordElem.send_keys(config.etsy_password)
# Submit sign in information
passwordElem.submit()
sleep(5)

In [16]:
#Create data frames to hold collected data
final_stats_df = pd.DataFrame(columns=['listingID',
    'fromDate',
    'toDate',
    'visits',
    'itemsSold',
    'revenue'])
final_search_terms_df=pd.DataFrame(columns=['searchTerms',
    'count','fromDate','toDate','listingID'])

#Loop through each listing
for listing in unique_listings:
    
    listing_stat_df = pd.DataFrame(columns=['listingID',
    'fromDate',
    'toDate',
    'visits',
    'itemsSold',
    'revenue'])
    
    search_terms_by_listing=pd.DataFrame(columns=['searchTerms',
    'count','fromDate','toDate','listingID'])
    

    #Create variables for date using info from data frame
    creation_year=df3.loc[df3.ListingID == listing, 's_year'].values[0]
    creation_month=df3.loc[df3.ListingID == listing, 's_month'].values[0]
    creation_day=df3.loc[df3.ListingID == listing, 's_day'].values[0]

    end_year=df3.loc[df3.ListingID == listing, 'e_year'].values[0]
    end_month=df3.loc[df3.ListingID == listing, 'e_month'].values[0]
    end_day=df3.loc[df3.ListingID == listing, 'e_day'].values[0]
    
    #function to call a list of dates between two dates, inclusive of start date, exclusive of end date
    def daterange(start_date, end_date):
        for n in range(int ((end_date - start_date).days)):
            yield start_date + timedelta(n)

    #Create list to hold date range and find date range
    dates = []
    start_date = date(creation_year, creation_month, creation_day)
    end_date = date(end_year,end_month,end_day)
    for single_date in daterange(start_date, end_date):
        dates.append(single_date.strftime("%Y-%m-%d"))
    
    #append listing id to list to append to data frame at end of loop
    listingID=[]
    listingID.append(listing)

    #Loop through each list of dates 
    for i in range(len(dates)):
        if 2 <= len(dates):
            if i <= len(dates)-2:
                
                fromDate=[]
                toDate=[]
                
                fromdate=dates[i]
                todate=dates[i+1]
                
                fromDate.append(fromdate)
                toDate.append(todate)
                                
                query_url=(f'https://www.etsy.com/your/shops/me/stats/listings/{listing}?end_date={todate}&start_date={fromdate}')
                driver.get(query_url)
                sleep(2)
                
                html=driver.page_source
                soup = BeautifulSoup(html, 'html.parser')
                
#                 #click on button to access drop down list
#                 element = driver.find_element_by_xpath('//*[contains(@id,"_button")]')
#                 actions = ActionChains(driver)
#                 actions.move_to_element(element).click().perform()
#                 sleep(2)
#                 #click on on 'custom'
#                 download = driver.find_element_by_xpath('//*[contains(@id,"_10")]')
#                 actions = ActionChains(driver)
#                 actions.move_to_element(download).click().perform()
#                 sleep(2)
#                 #enter from date
#                 element = driver.find_element_by_xpath('//*[contains(@id,"theseus-overlay-")]/div/div[2]/div/div[1]/div[1]/label/input')
#                 actions = ActionChains(driver)
#                 actions.move_to_element(element).click().perform()
#                 element.clear()
#                 actions.send_keys(fromdate).perform()
#                 #enter to date
#                 element = driver.find_element_by_xpath('//*[contains(@id,"theseus-overlay-")]/div/div[2]/div/div[1]/div[2]/label/input')
#                 actions = ActionChains(driver)
#                 actions.move_to_element(element).click().perform()
#                 element.clear()
#                 actions.send_keys(todate).perform()
#                 sleep(1)
#                 #click apply button
#                 element = driver.find_element_by_xpath('//*[contains(@id,"theseus-overlay-")]/div/div[3]/div/button[1]')
#                 actions = ActionChains(driver)
#                 actions.move_to_element(element).click().perform()
#                 sleep(2)
                
                #Create list to hold information collected
        
                visits=[]
                itemsSold=[]
                revenue=[]

                html=driver.page_source
                soup = BeautifulSoup(html, 'html.parser')
                visit_location=soup.find_all('div',class_='block-grid-item')

                counter=0
                #Collect data
                for i in visit_location:

                    visit=i.find('div',class_='col-xs-7 pl-xs-0 pr-xs-0 text-right').text
                    if counter == 0:
                        visit=visit.replace(' visits','').replace(' visit','')
                        visits.append(visit)
                    if counter == 1:
                        visit=visit.replace(' items sold','').replace(' item sold','')
                        itemsSold.append(visit)
                    if counter == 2:
                        visit=visit.replace('$','')
                        revenue.append(visit)
                    counter += 1
                
                #Create data frame with data
                stats_df = pd.DataFrame({'listingID':listingID,
                    'fromDate':fromDate,
                    'toDate':toDate,
                    'visits':visits,
                    'itemsSold':itemsSold,
                    'revenue':revenue})
            
                #Append data frame to master data frame
                final_stats_df=final_stats_df.append(stats_df,ignore_index=True, sort=True)
                
                listing_stat_df=listing_stat_df.append(stats_df,ignore_index=True,sort=True)
        #if there aren't enough dates print statement
        else:
            print(f'not enough dates{listing}')

    #function to call weekly dates from start date
    def perdelta(start, end, delta):
        curr = start
        while curr < end:
            yield curr
            curr += delta
    
    #Create list to hold date range and find date range
    weekly_dates = []
    start_date = date(creation_year, creation_month, creation_day)
    end_date = date(end_year,end_month,end_day)
    
    #run function
    for result in perdelta(start_date, end_date, timedelta(days=7)):
        weekly_dates.append(result.strftime("%Y-%m-%d"))
    
    #Loop through each list of dates 
    for i in range(len(weekly_dates)):
        if 2 <= len(weekly_dates):
            if i <= len(weekly_dates)-2:
                
                
                fromdate=weekly_dates[i]
                todate=weekly_dates[i+1]
                

                html=driver.page_source
                soup = BeautifulSoup(html, 'html.parser')
                
                

                query_url=(f'https://www.etsy.com/your/shops/me/stats/listings/{listing}?end_date={todate}&start_date={fromdate}')
                driver.get(query_url)
                sleep(2)
                
#                 #Enter dates
#                 element = driver.find_element_by_xpath('//*[contains(@id,"_button")]')
#                 actions = ActionChains(driver)
#                 actions.move_to_element(element).click().perform()
#                 sleep(2)
#                 download = driver.find_element_by_xpath('//*[contains(@id,"_10")]')
#                 actions = ActionChains(driver)
#                 actions.move_to_element(download).click().perform()
#                 sleep(2)
#                 element = driver.find_element_by_xpath('//*[contains(@id,"theseus-overlay-")]/div/div[2]/div/div[1]/div[1]/label/input')
#                 actions = ActionChains(driver)
#                 actions.move_to_element(element).click().perform()
#                 element.clear()
#                 actions.send_keys(fromdate).perform()
                
#                 element = driver.find_element_by_xpath('//*[contains(@id,"theseus-overlay-")]/div/div[2]/div/div[1]/div[2]/label/input')
#                 actions = ActionChains(driver)
#                 actions.move_to_element(element).click().perform()
#                 element.clear()
#                 actions.send_keys(todate).perform()
#                 sleep(1)
#                 element = driver.find_element_by_xpath('//*[contains(@id,"theseus-overlay-")]/div/div[3]/div/button[1]')
#                 actions = ActionChains(driver)
#                 actions.move_to_element(element).click().perform()
#                 sleep(2)
                
                search_terms_list=[]
                search_terms_list2=[]
                search_terms_list3 = []
                search_terms_list4=[]
                search_terms_count=[]

                html=driver.page_source
                soup = BeautifulSoup(html, 'html.parser')


                soup=BeautifulSoup(html,'lxml')
                #collect search term data
                table=soup.find_all('table')[0]

                tr_data=table.find_all('tr')

                for tr in tr_data[1:]:
                    td = tr.find_all('td')
                    row = [tr.text for tr in td]
                    search_terms_list.append(row)


                #turn list of lists into a single list
                for sublist in search_terms_list:
                    for item in sublist:
                        search_terms_list2.append(item)

                import re



                # split: \d is a digit (a character in the range 0-9), and + means 1 or more times. So, \d+ is 1 or more digits.
                for l in search_terms_list2:
                    simple=re.split('(\d+)',l)
                    search_terms_list3.append(simple[0])
                    search_terms_count.append(simple[-2])




                for string in search_terms_list3:
                    string = re.sub('[^a-zA-Z.\d\s]', '', string)
                    search_terms_list4.append(string)

                #remove leading and trailing spaces
                search_terms_list4 = [x.strip(' ') for x in search_terms_list4]

                search_terms_df=pd.DataFrame({"searchTerms":search_terms_list4,
                "count":search_terms_count})

                search_terms_df['fromDate']=fromdate
                search_terms_df['toDate']=todate

                search_terms_df['listingID']=listing            
                
                final_search_terms_df=final_search_terms_df.append(search_terms_df,ignore_index=True, sort=True)
                
                search_terms_by_listing=search_terms_by_listing.append(stats_df,ignore_index=True,sort=True)

        else:
            print(f'not enough dates{listing}')
    
    
    print(f'{listing} Complete')
    listing_stat_df.to_csv(f"/Users/jessicaetchechury/Desktop/JEtchCreationsDashboard/Historical_Data/historical_listing_data_{listing}_{today_m_d_y}.csv",index=False,header=True,encoding="UTF-8")
    search_terms_by_listing.to_csv(f"/Users/jessicaetchechury/Desktop/JEtchCreationsDashboard/Historical_Data/historical_search_terms_{listing}_{today_m_d_y}.csv",index=False,header=True,encoding="UTF-8")

final_stats_df.to_csv(f"/Users/jessicaetchechury/Desktop/JEtchCreationsDashboard/Historical_Data/historical_listing_data_{today_m_d_y}.csv",index=False,header=True,encoding="UTF-8")
final_search_terms_df.to_csv(f"/Users/jessicaetchechury/Desktop/JEtchCreationsDashboard/Historical_Data/historical_search_terms__by_listing_data_{today_m_d_y}.csv",index=False,header=True,encoding="UTF-8")




691796412 Complete
680069172 Complete
680085112 Complete
680082346 Complete
700382982 Complete
702303569 Complete
690717354 Complete
680073964 Complete
680077296 Complete
685451744 Complete
701344590 Complete
688443618 Complete
701210678 Complete
701339360 Complete
680079806 Complete
693930997 Complete
693947901 Complete
701202082 Complete
680071910 Complete
680064082 Complete
704569159 Complete
688465922 Complete
700377146 Complete
680084324 Complete
693933847 Complete
706002567 Complete
703387331 Complete
701341356 Complete
687898920 Complete


NameError: name 'final_df' is not defined